In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
from dateutil import parser
import re
import numpy as np

In [2]:
import spacy
import pytextrank
 
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe("textrank")

C:\Users\agata\AppData\Local\Programs\Python\Python312\Lib\site-packages


In [ ]:
pd.set_option('display.max_colwidth', 100)

In [3]:
df = pd.read_csv("scraped_articles_all.csv")

In [4]:
df_uktn = df[df['source'] == 'UKTN']

In [5]:
df = df[df.source != 'UKTN']

In [6]:
df_uktn['content'] = df_uktn['content'].str.split(r'\bTopics[^ ;\n]*').str[0]
df_uktn

,title,published_date,author,content,category,source
9356,PQShield: Startup developing quantum-proof cyb...,2024-06-20,Oscar Hornstein,"PQShield, an Oxford-based cybersecurity compan...",News,UKTN
9357,Anglo-French VC Breega launches Africa fund to...,2024-06-20,Robert Scammell,Anglo-French VC firm Breega has launched a $75...,News,UKTN
9358,Podcast: Reflecting on Tech Nation’s legacy – ...,2024-06-18,Robert Scammell,"In this episode of the UKTN Podcast, Founders ...",News,UKTN
9359,Dorset startup secures £3.6m to clean equipmen...,2024-06-17,Oscar Hornstein,"Spotless Water, a Dorset-based startup that su...",News,UKTN
9360,London stock market overtakes Paris as Raspber...,2024-06-17,Oscar Hornstein,"The London markets, still celebrating the succ...",News,UKTN
...,...,...,...,...,...,...
10208,UK-based Molten Ventures leads $25m bet in syn...,2022-01-11,Shubham Sharma,UK-based venture capital firm Molten Ventures ...,News,UKTN
10209,North East Venture Fund backs startup bringing...,2022-01-11,Shubham Sharma,Gateshead-based startup ART Health Solutions h...,News,UKTN
10210,Starling Bank shuns Facebook over scam ads,2022-01-07,Robert Scammell,British challenger bank Starling has stopped a...,News,UKTN
10211,London’s iProov secures £52m for online facial...,2022-01-06,Staff Reporter,"iProov, a London-based cybersecurity company t...",News,UKTN


In [7]:
display(df_uktn.loc[df_uktn['title'] == 'Courier sues Uber Eats over ‘racist’ facial recognition dismissal']['content'])

10018    A former Uber Eats courier has brought legal a...
Name: content, dtype: object

In [8]:
df = pd.concat([df, df_uktn])

In [9]:
df.drop_duplicates()
df = df.drop_duplicates(['title'], keep='first')

In [23]:
keywords = [
'workforce diversity',
'diversity in workplace',
'workplace diversity',
'workplace discrimination',
'discrimination at work',
'discrimination',
'fighting racism']

In [24]:
def search(regex: str, df, case=False):
    """Search all the text columns of `df`, return rows with any matches."""
    textlikes = df[["title", "content"]]
    return df[
        textlikes.apply(
            lambda column: column.str.contains(regex, regex=True, case=case, na=False)
        ).any(axis=1)
    ]

In [25]:
keywords_df = pd.DataFrame(columns=["title", "published_date", "author", "content", "category", "source"] )

In [26]:
appended_data = []
for keyword in keywords:
    appended_data.append(search(keyword, df))
    # keywords_df = pd.concat([search(keyword, df)], ignore_index = True, sort=False)
keywords_df = pd.concat(appended_data)


In [27]:
keywords_df

,title,published_date,author,content,category,source
3316,The legal landscape of HR compliance for startups,2024-01-01,Nancy Steidl,"The legal infrastructure surrounding Human Resources (HR) is one of the most important aspects, ...",HR,Startups Magazine
3344,The Flexible Working Bill: what you need to know,2024-01-01,Anna Wood,"On the 6th April, the work landscape in the UK will be undergoing a big change. The date marks t...",HR,Startups Magazine
3973,Businesses embrace strategies to rewrite code on diversity in tech,2024-01-01,Anna Wood,Businesses are making positive strides towards overcoming the historical lack of diversity withi...,Diversity,Startups Magazine
4011,Why black representation matters,2023-01-01,Ashleigh Ainsley,"Take the time to do some research, and you can quickly find the names of 50 leading UK-based bla...",Diversity,Startups Magazine
4035,The business benefit of diverse teams,2022-01-01,David Rai,"Promoting diversity and equality is not just a socially responsible initiative, but a strategic ...",Diversity,Startups Magazine
...,...,...,...,...,...,...
9051,BAME & LGBTQI+ Youths Most At Risk For Online Harm,2022-03-22,TechRound Team,New research from the University of Surrey finds BAME and LGBTQI+ youths at most risk of harm on...,Business,TechRound
9137,4 Top Tips To Use AI Ethically & Responsibly in 2022,2022-03-02,TechRound Team,AI (artificial intelligence) is an amazing tool to improve efficiency and speed up processes but...,Business,TechRound
9303,Connectr Reveals The Five Groups the Pandemic Hit Hardest,2022-01-20,TechRound Team,"Research by Connectr, the leading HR tech platform for building businesses where people belong, ...",Business,TechRound
9681,Data watchdog sounds neurotech discrimination warning,2023-06-08,Oscar Hornstein,The UK data watchdog has warned that neurotech has a “real danger” of bias and discrimination if...,News,UKTN


In [29]:
keywords_df.to_csv("diversity_articles_short.csv", index = None, header=True) 
